In [1]:
import numpy as np
import copy

with open("input.txt", 'r') as f:
    input = f.read()

# solution: 7036
input1 = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

# solution: 11084
input1 = """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""

def print_maze(maze):
    print("\n".join(["".join(row.tolist()) for row in maze]))

maze = np.array([list(row) for row in input.split("\n")])
end = int(np.where(maze == "E")[0]), int(np.where(maze == "E")[1])
start = int(np.where(maze == "S")[0]), int(np.where(maze == "S")[1])

/var/folders/6c/gtmx1v3j1m544q5skcd8r6wh0000gr/T/ipykernel_66191/519590573.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  end = int(np.where(maze == "E")[0]), int(np.where(maze == "E")[1])
/var/folders/6c/gtmx1v3j1m544q5skcd8r6wh0000gr/T/ipykernel_66191/519590573.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  start = int(np.where(maze == "S")[0]), int(np.where(maze == "S")[1])


In [2]:
# note that we cannot rotate two times in a row

initial_dir = ">"

DIRS_TO_STR = {
    0: "^",
    1: ">",
    2: "v",
    3: "<"
}
DIRS_TO_INT = {v:k for k,v in DIRS_TO_STR.items()}

DIRECTION = {
    "^": (-1,0),
    ">": (0,1),
    "v": (1,0),
    "<": (0,-1)
}

ROTATE_CLOCKWISE = {
    "^": ">",
    ">": "v",
    "v": "<",
    "<": "^"
}

def rotate(dir: str, clockwise=True):
    if clockwise:
        return ROTATE_CLOCKWISE[dir]
    else:
        return ROTATE_CLOCKWISE[ROTATE_CLOCKWISE[ROTATE_CLOCKWISE[dir]]]

def solve_maze_iter(maze: np.ndarray, start: tuple, dir: str, end: tuple):
    costs = np.ones((maze.shape[0], maze.shape[1], 4))* float("inf")
    next_steps = [(start[0], start[1], DIRS_TO_INT[dir])]
    costs[start[0], start[1], DIRS_TO_INT[dir]] = 0
    
    while len(next_steps) > 0:
        state = next_steps[0]
        pos = state[0], state[1]
        dir = DIRS_TO_STR[state[2]]
        running_cost = costs[state]
        for d in [dir, rotate(dir, clockwise=True), rotate(dir, clockwise=False)]:

            new_pos = pos[0] + DIRECTION[d][0], pos[1] + DIRECTION[d][1]
            if maze[new_pos] == "#": continue
            if d == dir: dir_cost = 0
            else: dir_cost = 1000
            current_cost = running_cost + dir_cost + 1
            new_state = (new_pos[0], new_pos[1], DIRS_TO_INT[d])
            if current_cost < costs[new_state]:
                costs[new_state] = running_cost + dir_cost + 1
                next_steps.append(new_state) # add new items to explore
        next_steps = next_steps[1:] # remove the item we just explored
    return costs

costs = solve_maze_iter(maze, start, initial_dir, end)
min_cost = int(np.min(costs[end[0],end[1]]))
min_cost

109516

### Part 2

In [3]:
# this takes an insane amount of time but works            
def good_positions_divide(maze, start, end, costs, min_cost):
    # divide the path into start -> x -> end and measure cost separtely for start->x and x->end
    best = np.zeros_like(maze, dtype=int)
    new_maze = copy.copy(maze)
    # print(maze.shape)
    for i in range(maze.shape[0]):
        print(f"row{i}")
        for j in range(maze.shape[1]):
            if maze[i,j] == "#": continue
            # costs1 = solve_maze_iter(maze, start, initial_dir, (i,j))
            min_cost1 = (np.min(costs[i,j]))
            # print(costs[i,j], costs1[i,j])
            possible_dirs = [l[0] for l in np.argwhere(costs[i, j] == min_cost1).tolist()]
            for dir in possible_dirs:
                dir_s = DIRS_TO_STR[dir]
                costs2 = solve_maze_iter(maze, (i,j), dir_s, end)
                min_cost2 = (np.min(costs2[end[0],end[1]]))
                if min_cost1 + min_cost2 == min_cost:
                    best[i,j] = 1
                    new_maze[i,j] = "O"
                    # break
    return best, new_maze

            

best, new_maze = good_positions_divide(maze, start, end, costs, min_cost)

print_maze(new_maze)
print(best)
print(len(np.argwhere(new_maze == "O")))
np.sum(best)
# print_maze(best)

row0
row1
row2
row3
row4
row5
row6
row7
row8
row9
row10
row11
row12
row13
row14
row15
row16
row17
row18
row19
row20
row21
row22
row23
row24
row25
row26
row27
row28
row29
row30
row31
row32
row33
row34
row35
row36
row37
row38
row39
row40
row41
row42
row43
row44
row45
row46
row47
row48
row49
row50
row51
row52
row53
row54
row55
row56
row57
row58
row59
row60
row61
row62
row63
row64
row65
row66
row67
row68
row69
row70
row71
row72
row73
row74
row75
row76
row77
row78
row79
row80
row81
row82
row83
row84
row85
row86
row87
row88
row89
row90
row91
row92
row93
row94
row95
row96
row97
row98
row99
row100
row101
row102
row103
row104
row105
row106
row107
row108
row109
row110
row111
row112
row113
row114
row115
row116
row117
row118
row119
row120
row121
row122
row123
row124
row125
row126
row127
row128
row129
row130
row131
row132
row133
row134
row135
row136
row137
row138
row139
row140
###########################################################################################################################

np.int64(568)